# Condition Variables

We often need to have a thread wait for some condition to be true before it can continue executing.

One thread might need to wait for another thread to complete some action before it can continue with its execution.

A similar (but not exact) example is a parent waiting for a child.

We accomplish this with **condition variables**.

A **condition variable** is a queue of waiting threads.

We always need a lock associated with a condition variable, and we have two concurrency functions we use to interact with condition variables:

- `wait(cond_t *cv, lock_t *mutex)`
    - it is assumed that the thread calling wait holds the lock when it is called.
    - puts the caller to sleep, releasing the lock.
    - when woken, `wait()` then reaquires the lock and returns, allowing this thread to continue
- `signal(cond_t *cv)`
    - signals that one thread waiting on the condition variable can wake up and execute

